In [25]:
import torch
from torch import nn, optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import tqdm
from torch.nn import functional as F
from sklearn.metrics import accuracy_score
import warnings

import numpy as np 
from IPython.display import clear_output
from tqdm.auto import tqdm, trange

In [47]:
train_root = '../input/homework1/train'
val_root = '../input/homework1/val'

# grayscale = transforms.Grayscale(num_output_channels=1)
# rotation = transforms.RandomRotation(degrees=(-8, 8), expand=True)

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
rs = transforms.Resize((224, 224))

train_transform = transforms.Compose([rs,
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(degrees=(-8, 8), expand=False),
        transforms.ToTensor(),
        normalize,
    ])
val_transform = transforms.Compose([rs, 
        transforms.ToTensor(),
        normalize,
])

trainset = torchvision.datasets.ImageFolder(train_root, transform=train_transform)
valset = torchvision.datasets.ImageFolder(val_root, transform=val_transform)

In [27]:
# Just very simple sanity checks
assert isinstance(trainset[0], tuple)
assert len(trainset[0]) == 2
assert isinstance(trainset[1][1], int)
print("tests passed")

tests passed


In [28]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [52]:
# batch size 
batch_size = 64
num_workers = 2
train_dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True,  num_workers=num_workers)
val_dataloader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [53]:
import torchvision.models as models

n_epochs = 20
model = models.resnet50(pretrained=True)  # TODO resnet18
model.fc = nn.Linear(model.fc.in_features, 200)

optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)  # TODO was 0.02
criterion = nn.CrossEntropyLoss()

# scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20, eta_min=0, last_epoch=- 1, verbose=True)
# model.fc.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.25, training=m.training))  # TODO

model.to(device);

In [54]:
def train_one_epoch(model, train_dataloader, criterion, optimizer, device="cuda:0"):
    train_epoch_loss, train_epoch_true_hits = torch.empty(0).to(device), torch.empty(0)
    train_predicted_labels, train_real_labels, train_losses = [], [], []
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []
    model.train()
    
    for i, (imgs, labels) in enumerate(train_dataloader):
        imgs, labels = imgs.to(device), labels.to(device)
        y_pred = model(imgs)
        train_epoch_loss = train_epoch_loss#.to(device)
        train_epoch_true_hits = train_epoch_true_hits#.to(device)
        loss = criterion(y_pred, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        train_epoch_loss = torch.cat((train_epoch_loss, loss.unsqueeze(0) / labels.size(0)))
        train_loss_log.append(loss.data / labels.size(0))
        # log accuracy for the current epoch and the whole training history
        pred_classes = torch.argmax(y_pred, dim=-1)
        train_predicted_labels += pred_classes.tolist()
        train_real_labels += labels.tolist()
        
    return train_loss_log, train_predicted_labels, train_real_labels                  

def predict(model, val_dataloder, criterion, device="cuda:0"):
    all_predicted_labels, all_real_labels, all_losses =  [], [], []
    val_accuracy = []
    model.eval()   
    with torch.no_grad():
        for imgs, labels in val_dataloader:
            imgs, labels = imgs.to(device), labels.to(device)
            y_pred = model(imgs)

            for i in range(len(y_pred)):
                all_losses.append(criterion(y_pred[i], labels[i]).tolist())
            pred_classes = torch.argmax(y_pred, dim=-1)
            all_predicted_labels += pred_classes.tolist()
            all_real_labels += labels.tolist()
    return all_losses, all_predicted_labels, all_real_labels                  

def train(model, train_dataloader, val_dataloader, criterion, optimizer, device="cuda:0", n_epochs=10, scheduler=None):
    model.to(device)
    for epoch in range(n_epochs):
        t_los, t_acc, t_lab = train_one_epoch(model, train_dataloader, criterion, optimizer, device)
        train_accuracy = accuracy_score(t_acc, t_lab)
        v_los, v_acc, v_lab = predict(model, val_dataloader, criterion, device)
        val_accuracy = accuracy_score(v_acc, v_lab)
        print(f'{epoch}/{n_epochs}')
        #scheduler.step()  # TODO
        print("Train acc:", train_accuracy)
        print("Val acc:", val_accuracy)

In [55]:
train(model, train_dataloader, val_dataloader, criterion, optimizer, device, n_epochs, scheduler=None)  # TODO scheduler

0/20
Train acc: 0.54017
Val acc: 0.5891
1/20
Train acc: 0.65462
Val acc: 0.6174
2/20
Train acc: 0.70799
Val acc: 0.6562
3/20
Train acc: 0.74655
Val acc: 0.6685
4/20
Train acc: 0.78532
Val acc: 0.6666
5/20
Train acc: 0.81394
Val acc: 0.6774
6/20
Train acc: 0.83868
Val acc: 0.6859
7/20
Train acc: 0.8639
Val acc: 0.6943
8/20
Train acc: 0.88098
Val acc: 0.6868
9/20
Train acc: 0.90012
Val acc: 0.6848
10/20
Train acc: 0.91357
Val acc: 0.6896
11/20
Train acc: 0.92751
Val acc: 0.6935
12/20
Train acc: 0.9369
Val acc: 0.6972
13/20
Train acc: 0.94877
Val acc: 0.6943
14/20
Train acc: 0.95642
Val acc: 0.6995
15/20
Train acc: 0.96168
Val acc: 0.703
16/20
Train acc: 0.96799
Val acc: 0.7091
17/20
Train acc: 0.9715
Val acc: 0.7068
18/20
Train acc: 0.97327
Val acc: 0.7104
19/20
Train acc: 0.97808
Val acc: 0.7067


In [56]:
loss, pred_labels, real_labels = predict(model, val_dataloader, criterion, device)

In [57]:
len(pred_labels) == len(valset)

True

In [58]:
accuracy_score(real_labels, pred_labels)

0.7067

In [59]:
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(valset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за это задание составит {} баллов".format(min(10, 10 * (accuracy - 0.5) / 0.34)))

Оценка за это задание составит 6.079411764705882 баллов


In [61]:
# Что если сделать еще пару трейн эпох?
train(model, train_dataloader, val_dataloader, criterion, optimizer, device, n_epochs, scheduler=None)  # TODO scheduler

0/20
Train acc: 0.98008
Val acc: 0.7131
1/20
Train acc: 0.98317
Val acc: 0.7157
2/20
Train acc: 0.98441
Val acc: 0.7153


KeyboardInterrupt: 

In [62]:
# После небольшого дообучения модели. Дообучения прервала самостоятельно, так как обучение занимает много времени
all_losses, predicted_labels, true_labels = predict(model, val_dataloader, criterion, device)
assert len(predicted_labels) == len(valset)
accuracy = accuracy_score(true_labels, predicted_labels)
print("Оценка за это задание составит {} баллов".format(min(10, 10 * (accuracy - 0.5) / 0.34)))

Оценка за это задание составит 6.335294117647059 баллов


## Итоговый результат выше - Оценка за это задание составит 6.335294117647059 баллов

## Задание 2 (про улучшения https://datahacker.rs/016-pytorch-three-hacks-for-improving-the-performance-of-deep-neural-networks-transfer-learning-data-augmentation-and-scheduling-the-learning-rate-in-pytorch/)
Первые попытки дообучить модель из первого задания:
1) Возьмем предобученную модель **resnet18**, lr = 0.01, batch = 50, norm, rotate, flip из предыдущего задания (но включили нормировку). Получилось максимальное качество 50.3%

2) **Resnet18** без нормализации. Получаем 50% на 9 эпохе при train_accuracy = 0.7. Запустим 20 эпох при lr=0.015, лучший результат 51.2 на 14 эпохе, время обучения очень долгое последней эпохи

2.0) **Resnet18** без нормализации. Обучила 50 эпох, после 24 качество начало заметно ухудшаться. Максимум - 51%, он достигается уже на 15 эпохе. Был добавлен dropout для теста model.fc.register_forward_hook(lambda m, inp, out: F.dropout(out, p=0.5, training=m.training)). После 20й эпохи качество начало резко ухудшаться и упало до 44%, потом не увеличивалось. Возможно 0.5 - это слишком много

2.1) Модель **resnet 50** - максимальное качество 54% на 5 эпохе, все по-старому, оставила dropout, хоть train_acc = 74, модель топчется на одном месте, lr = 0.02 велик, а dropout 0.5 - это слишком много

2.2) Модель **resnext50 - 32-4d** просто проверить. Dropout = 0.25, lr = 0.02, batch = 100 - очень долго, остановила

3) Вернусь к обычной **resnet-50**, batch = 100, dropout off, lr = 0.02 - accuracy 0.554 на 6 эпохе

3.1) **resnet-50**. Изменила rotation на -8, +8 + expand=False, 50% качества на 2й эпохе,  56.14% на 7й эпохе

3.2) **resnet-50**. Изменила rotation на -8, +8 + expand=True, 48% качества на 2й эпохе, 56.3 % на 7й эпохе, 57 на 10й acc_train = 92. Если и expand=True дает эффект, то не очень очевидный

## Задание 2 продолжение:
**Воспользуемся отменой на ограничение ресайза картинок. 
Из документации предобученных моделей: https://pytorch.org/vision/stable/models.html и кода: https://github.com/pytorch/examples/blob/42e5b996718797e45c46a25c55b031e6768f8440/imagenet/main.py#L89-L101
имплементируем нормализацию transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) и нужный размер для ресайза. Сделаем ресайз сразу до нужного размера (224х224), обучим resnet-50.
**

1) Добавлен ресайз картинок как в описании pytorch для предобученной модели resnet50 до размеров, принимаемых на вход по умолчанию - 224х224. Получили качество 67% при ресайзе картинок на первых 2х эпохах без ротации картинки и без шедулера. 

2) Без шедулера, но с ротацией картинок на 2й эпохе 59%, на 7й эпохе 65. Lr = 0.02 - очень большой, обучение очень медленное, качество быстро перестает расти и топчется несколько эпох на одном месте

3) Поставим lr = 0.01, С таким lr качество на 2й эпохе 65. Кажется, что так эффективнее. Шедулер отключен. Обучим 20+ эпох в поисках высокого качества. На 20ти эпохах максимум 71% качества. 
Попробуем чуть-чуть дообучить модель при тех же параметрах 

